In [ ]:
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal

import tensorflow as tf
from tensorflow.keras.layers import InputLayer
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Model

from tensorflow.keras.utils import plot_model
from os import listdir
from numpy import asarray, load
from numpy import vstack
from keras_preprocessing.image import img_to_array
from keras_preprocessing.image import load_img
from numpy import savez_compressed
from matplotlib import pyplot as plt
import numpy as np
from keras import Input

from keras.metrics import MeanIoU
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Cropping2D

def define_encoder_block(layer_in, n_filters, resnet_model, prefix='encoder_'):
    # Use the output of the last residual block as the output of the encoder block
    resnet_output = resnet_model.get_layer('conv5_block3_out').output

    # Additional convolutional layer for adapting the output channels to n_filters
    conv = Conv2D(n_filters, (1, 1), strides=(1, 1), padding='same', name=prefix+'conv_adapt')(resnet_output)

    # Batch normalization and LeakyReLU activation
    conv = BatchNormalization(name=prefix+'batch_norm')(conv)
    encoder_block_output = LeakyReLU(alpha=0.2, name=prefix+'leaky_relu')(conv)
    return encoder_block_output

from tensorflow.keras.layers import Cropping2D


from tensorflow.keras.layers import UpSampling2D

from tensorflow.keras.layers import Conv2DTranspose

from tensorflow.keras.layers import UpSampling2D

def decoder_block(layer_in, skip_in, n_filters, dropout=True, prefix='decoder_'):
    init = RandomNormal(stddev=0.02)
    print("enter decoder")
    print(layer_in)
    # Upsample using UpSampling2D
    #g = UpSampling2D(size=(2, 2), interpolation='nearest', name=prefix+'upsampling')(layer_in)
    #print(g)
    g = Conv2D(n_filters, (3, 3), strides=(1, 1), padding='same', kernel_initializer=init, name=prefix+'conv')(layer_in)
    print(g)
    g = BatchNormalization(name=prefix+'batch_norm')(g, training=True)
    #print(g)
    # Calculate cropping values
    crop_size_y = max(0, (skip_in.shape[1] - g.shape[1]) // 2)
    crop_size_x = max(0, (skip_in.shape[2] - g.shape[2]) // 2)

    # Crop and concatenate with skip connection from the corresponding encoder block
    skip_cropped = Cropping2D(cropping=((crop_size_y, crop_size_y), (crop_size_x, crop_size_x)))(skip_in)
    g = Concatenate(name=prefix+'concatenate')([g, skip_cropped])

    # Optional dropout layer
    if dropout:
        g = Dropout(0.5, name=prefix+'dropout')(g, training=True)

    # Apply activation function (ReLU)
    g = Activation('relu', name=prefix+'activation')(g)
    print(g)
    return g


def define_unet_resnet50(resnet_model, image_shape=(256, 256, 3)):
    init = RandomNormal(stddev=0.02)

    # Encoder
    for layer in resnet_model.layers:
        layer.trainable = False  # Freeze ResNet layers

    e1 = resnet_model.get_layer('conv1_relu').output
    e2 = resnet_model.get_layer('conv2_block3_out').output
    e3 = resnet_model.get_layer('conv3_block4_out').output
    e4 = resnet_model.get_layer('conv4_block6_out').output

    # Decoder
    d = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e4)
    d = Activation('relu')(d)
    d = Concatenate()([d, e3])
    d = Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = Activation('relu')(d)
    d = Concatenate()([d, e2])
    d = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = Activation('relu')(d)
    d = Concatenate()([d, e1])



    # Output layer
    out_layer = Conv2DTranspose(image_shape[2], (4, 4), strides=(2, 2), padding='same', kernel_initializer=init, activation='tanh', name='output_image')(d)
    out_image = Activation('tanh')( out_layer)
    model = Model(resnet_model.input, out_image, name='generator')
    #model.summary()

    return model

# Example usage
image_shape = (256, 256, 3)
# Create ResNet50 model without top layers
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=image_shape)

# Pass the ResNet50 model to the U-Net model
unet_resnet50_model = define_unet_resnet50(resnet_model, image_shape)
unet_resnet50_model.summary()
opt = Adam(learning_rate=0.0002, beta_1=0.5)
unet_resnet50_model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])

In [ ]:
from tensorflow.keras.losses import categorical_crossentropy
import tensorflow.keras.backend as K

def dice_coefficient(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
    return K.mean((2. * intersection + 1.) / (union + 1.))

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def combined_loss(y_true, y_pred):
    # Categorical Crossentropy Loss
    cat_cross_loss = categorical_crossentropy(y_true, y_pred)

    # Dice Loss
    dice_loss_value = dice_loss(y_true, y_pred)

    # Combine the losses (you can adjust the weights as needed)
    alpha = 0.5  # Weight for Categorical Crossentropy Loss
    beta = 0.5   # Weight for Dice Loss
    combined = alpha * cat_cross_loss + beta * dice_loss_value

    return combined



In [ ]:
from tensorflow.keras.layers import Input, Concatenate, Conv2D, LeakyReLU, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal

def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)

    in_src_image = Input(shape=image_shape)
    in_target_image = Input(shape=image_shape)

    # Concatenate source and target images
    merged = Concatenate()([in_src_image, in_target_image])

    # Convolutional layers with LeakyReLU activation and batch normalization
    d = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(512, (4, 4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(1, (4, 4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)

    # Define model
    model = Model([in_src_image, in_target_image], patch_out)

    # Compile the model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])

    return model

# Example usage
discriminator_model = define_discriminator(image_shape)
discriminator_model.summary()

In [ ]:
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input

# Combined generator and discriminator model for updating the generator
def define_gan(generator_model, discriminator_model, image_shape):

    input_image = Input(shape=image_shape)
    generated_image = generator_model(input_image)
    validity = discriminator_model([input_image, generated_image])

    model = Model(inputs=input_image, outputs=[validity, generated_image])

    model.compile(loss=['binary_crossentropy', 'mae'],
               optimizer=opt, loss_weights=[1,100])

    return model


In [ ]:
# Create ResNet50 model without top layers for the generator
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=image_shape)
generator_model = define_unet_resnet50(resnet_model, image_shape)

# Create discriminator model
discriminator_model = define_discriminator(image_shape)


# Define GAN model
gan_model = define_gan(generator_model, discriminator_model, image_shape)

# Print summary of the GAN model
gan_model.summary()


In [ ]:
def generate_real_samples(dataset, n_samples, patch_shape):
	trainA, trainB = dataset
	ix = randint(0, trainA.shape[0], n_samples)
	X1, X2 = trainA[ix], trainB[ix]
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

def generate_fake_samples(g_model, samples, patch_shape):
	X = g_model.predict(samples)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

In [ ]:
def summarize_performance(step, g_model, dataset, n_samples=3):

	[X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
	X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)

	X_realA = (X_realA + 1) / 2.0
	X_realB = (X_realB + 1) / 2.0
	X_fakeB = (X_fakeB + 1) / 2.0

	for i in range(n_samples):
		plt.subplot(3, n_samples, 1 + i)
		plt.axis('off')
		plt.imshow(X_realA[i])

	for i in range(n_samples):
		plt.subplot(3, n_samples, 1 + n_samples + i)
		plt.axis('off')
		plt.imshow(X_fakeB[i])

	for i in range(n_samples):
		plt.subplot(3, n_samples, 1 + n_samples*2 + i)
		plt.axis('off')
		plt.imshow(X_realB[i])

	filename1 = 'plot_%06d_691_Images.png' % (step+1)
	plt.savefig(filename1)
	plt.close()

	filename2 = 'model_%06d_600_Images.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))


In [ ]:
def train(d_model, g_model, gan_model, dataset, n_epochs=10 , n_batch=1):
	n_patch = d_model.output_shape[1]
	trainA, trainB = dataset
	bat_per_epo = int(len(trainA) / n_batch)
	n_steps = bat_per_epo * n_epochs
	for i in range(n_steps):
		[X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
		if (i+1) % (bat_per_epo * 25) == 0:
			summarize_performance(i, g_model, dataset)

In [ ]:
def load_mask_names(path):
    src_list = list()
    for filename in listdir(path):
        src_list.append(filename)
    src_list.sort()
    return src_list

In [ ]:
def load_image_names(path):
    tar_list = list()
    for filename in listdir(path):
        tar_list.append(filename)
    tar_list.sort()
    return tar_list

In [ ]:
def load_masks(path, src_list_names , size=(256,256)):
    src_list = list()
    for fn in src_list_names:
        for filename in listdir(path):
            if fn == filename:
                src_img = load_img(path + filename, target_size=size)
                src_img = img_to_array(src_img)
                src_list.append(src_img)
                break

    return asarray(src_list)

In [ ]:
from PIL import Image

In [ ]:
IMG_SIZE1 , IMG_SIZE2 = 256 , 256
def load_images(path, tar_list_names , size=(256,256)):
    tar_list = list()
    for fn in tar_list_names:
        for filename in listdir(path):
            if fn == filename:
                img = Image.open(path + filename)
                imarray = np.array(img, dtype = 'float')
                if imarray.shape[0] != IMG_SIZE1 or imarray.shape[1] != IMG_SIZE2:
                    nimg = img.resize((IMG_SIZE1,IMG_SIZE2))
                    imarray = np.array(nimg, dtype = 'float')
                tar_list.append(imarray)
                break
    return asarray(tar_list)

In [ ]:
path_train_images = 'path of labelled images'

tar_images_names = load_image_names(path_train_images)

In [ ]:
len(tar_images_names)

In [ ]:
tar_images = load_images(path_train_images , tar_images_names)

In [ ]:
type(tar_images)

In [ ]:
tar_images.dtype

In [ ]:
path_train_masks = 'path of the masks of labelled set'

src_images_names =  load_mask_names(path_train_masks)

In [ ]:
len(src_images_names)

In [ ]:
src_images = load_masks(path_train_masks , src_images_names)

In [ ]:
type(src_images)

In [ ]:
src_images.dtype

In [ ]:
print('Loaded: ', src_images.shape, tar_images.shape)

In [ ]:
n_samples = 3
for i in range(n_samples):
	plt.subplot(2, n_samples, 1 + i)
	plt.axis('off')
	plt.imshow(src_images[i+201].astype('uint8'))

for i in range(n_samples):
	plt.subplot(2, n_samples, 1 + n_samples + i)
	plt.axis('off')
	plt.imshow(tar_images[i+201].astype('uint8'))
plt.show()

In [ ]:
def preprocess_data(data):
	X1, X2 = data[0], data[1]

	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X1, X2]

In [ ]:
data = [src_images , tar_images]

In [ ]:
dataset = preprocess_data(data)

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
train(discriminator_model, generator_model, gan_model, dataset, n_epochs=100, n_batch=1)


In [ ]:
from keras.models import load_model
from numpy.random import randint

model_path = 'model_path'

model = load_model(model_path)

In [ ]:
# plot source, generated and target images
def plot_images(src_img, gen_img, tar_img):
	images = vstack((src_img, gen_img, tar_img))
	# scale from [-1,1] to [0,1]
	images = (images + 1) / 2.0
	titles = ['Source', 'Generated', 'Expected']
	# plot images row by row
	plt.figure(figsize = (40,4))
	for i in range(len(images)):
		# define subplot

		plt.subplot(1, 3 , 1 + i )

		plt.axis('off')
		plt.imshow(images[i])
		plt.title(titles[i])
	plt.show()

In [ ]:
from keras.models import load_model
from numpy.random import randint

model_path = '/home/semanticseg/Desktop/data_dlrsd/model_079100_600_Images.h5/'

model = load_model(model_path)

In [ ]:
path_train_images_test = 'path to unlabelled images'

tar_images_names_test = load_image_names(path_train_images_test)

In [ ]:
len(tar_images_names_test)

In [ ]:
tar_images_test = load_images(path_train_images_test , tar_images_names_test)

In [ ]:
path_train_masks_test = 'path to predicted masks of unlabelled'

src_images_names_test =  load_mask_names(path_train_masks_test)

In [ ]:
src_images_names_test = src_images_names_test[1:]

In [ ]:
len(src_images_names_test)

In [ ]:
src_images_test = load_masks(path_train_masks_test , src_images_names_test)

In [ ]:
print('Loaded: ', src_images_test.shape, tar_images_test.shape)

In [ ]:
data_test = [src_images_test , tar_images_test]

In [ ]:
dataset_test = preprocess_data(data_test)

In [ ]:
n_samples = 3
for i in range(n_samples):
	plt.subplot(2, n_samples, 1 + i)
	plt.axis('off')
	plt.imshow(src_images_test[13+i].astype('uint8'))

for i in range(n_samples):
	plt.subplot(2, n_samples, 1 + n_samples + i)
	plt.axis('off')
	plt.imshow(tar_images_test[13+i].astype('uint8'))
plt.show()

In [ ]:
IOU_GeneratedMask = dict()

In [ ]:
[X1 , X2] = dataset_test
Iou_test = 0
l = len(X1)
for i in range(0 , l):
    ix = np.array(i)
    ix = ix.reshape((1,))

    src_image, tar_image = X1[ix], X2[ix]
    gen_image = model.predict(src_image)

    gen_image = (gen_image + 1) / 2.0
    tar_image = (tar_image + 1) / 2.0

    plot_images(src_image, gen_image, tar_image)

    file_name = tar_images_names_test[i];

    num_classes = "no.of classes"
    IOU_keras = MeanIoU(num_classes=num_classes)
    temp = IOU_keras.update_state(tar_image, gen_image)
    Iou_test = (IOU_keras.result().numpy())
    IOU_GeneratedMask[file_name] = Iou_test
    print(Iou_test)

In [ ]:
IOU_GeneratedMask_Dict = sorted(IOU_GeneratedMask.items(), key=lambda x: x[1], reverse=False)

In [ ]:
IOU_GeneratedMask_Dict

In [ ]:
import shutil

In [ ]:
iou_list = list(IOU_GeneratedMask_Dict)

iou_values = np.array([item[1] for item in iou_list])
mean_iou = np.mean(iou_values)
std_dev_iou = np.std(iou_values)
for i, (name, iou) in enumerate(iou_list):
    if iou >= mean_iou + std_dev_iou or iou <= mean_iou - std_dev_iou:
        src_file = "path of that image in unlabelled" + name[0]
        dst_file = "path of most confident set of images" +  name[0]
        shutil.copy(src_file, dst_file)

In [ ]:
for i in src_images_names:
    src = 'path of predicted masks' + i
    dst = 'path of most confident set of masks' + i

    shutil.move(src, dst)